OBJETIVO DEL MODELO 1

Predecir:

Total de delitos por municipio y mes teniendo en cuenta la suma de:

1. Hurtos
2. Violencia intrafamiliar
3. Delitos sexuales
4. Lesiones personales
5. Incautaciones de droga
6. Delitos contra menores

#### Librerías y dependencias

In [1]:
import xgboost as xgb
print(xgb.__version__)

3.1.2


In [2]:
import os, json
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Preparación de datos para el modelo

#### Normalización de delitos por mes

In [4]:
def normalizar_delito(
    path,
    delito_categoria,
    col_fecha='fecha_hecho',
    col_cantidad='cantidad',
    col_dane='codigo_dane',
    col_municipio='municipio'
):
    df = pd.read_csv(path)

    # DANE municipio
    df['codigo_dane_5d'] = df[col_dane].astype(str).str[:5]

    # Fecha: año / mes
    df[col_fecha] = pd.to_datetime(df[col_fecha], errors='coerce')
    df['anio'] = df[col_fecha].dt.year
    df['mes'] = df[col_fecha].dt.month

    df['delito_categoria'] = delito_categoria

    # CASO DROGAS
    if delito_categoria == "DROGA":
        df['drogas_gramos'] = pd.to_numeric(df[col_cantidad], errors='coerce').fillna(0)
        df['cantidad'] = 1  # evento
    else:
        df['cantidad'] = pd.to_numeric(df[col_cantidad], errors='coerce').fillna(1)
        df['drogas_gramos'] = 0

    return df[['codigo_dane_5d', col_municipio, 'anio', 'mes',
               'delito_categoria', 'cantidad', 'drogas_gramos']]



In [5]:
hurtos_1 = normalizar_delito('/content/drive/MyDrive/atlas-del-crimen/raw/hurto_1.csv', 'HURTO')
hurtos_2 = normalizar_delito('/content/drive/MyDrive/atlas-del-crimen/raw/hurto_2.csv', 'HURTO')
hurtos_3 = normalizar_delito('/content/drive/MyDrive/atlas-del-crimen/raw/hurto_3.csv', 'HURTO')
hurtos_4 = normalizar_delito('/content/drive/MyDrive/atlas-del-crimen/raw/hurto_4.csv', 'HURTO')
drogas = normalizar_delito('/content/drive/MyDrive/atlas-del-crimen/raw/incautacion_drogas.csv', 'DROGA')
lesiones = normalizar_delito('/content/drive/MyDrive/atlas-del-crimen/raw/lesiones_personales.csv', 'LESIONES')
vif = normalizar_delito('/content/drive/MyDrive/atlas-del-crimen/raw/violencia_intrafamiliar.csv', 'VIF')
sexuales = normalizar_delito('/content/drive/MyDrive/atlas-del-crimen/raw/delitos_sexuales.csv', 'SEXUAL')
infancia = normalizar_delito('/content/drive/MyDrive/atlas-del-crimen/raw/delitos_infantes.csv', 'INFANCIA', col_dane='codigo_dane')
sexuales = normalizar_delito('/content/drive/MyDrive/atlas-del-crimen/raw/delitos_sexuales.csv', 'SEXUAL')


#### Tabla maestra de delitos (mensual)

In [6]:
crime_long = pd.concat([hurtos_1, hurtos_2, hurtos_3, hurtos_4, drogas, lesiones, vif, sexuales, infancia])
crime_mensual = (
    crime_long
    .groupby(['codigo_dane_5d', 'anio', 'mes', 'delito_categoria'], as_index=False)
    .agg({'cantidad':'sum'})
)
crime_long['cantidad'].unique()

array([ 1,  7,  2,  9,  6,  4,  8,  3,  5, 10, 11, 13, 12, 14, 15, 17, 16,
       18, 19, 20, 23, 24, 21, 22, 25, 29, 27, 26, 30, 28, 34, 32, 35, 33,
       38, 39, 45, 40, 31, 36, 58, 47, 42, 41, 37, 43, 44, 46, 48, 61, 54,
       55])

In [7]:
crime_long.head()

,codigo_dane_5d,municipio,anio,mes,delito_categoria,cantidad,drogas_gramos
0,68615,RIONEGRO,2014,9,HURTO,1,0.0
1,68190,CIMITARRA,2013,5,HURTO,1,0.0
2,68081,BARRANCABERMEJA,2025,3,HURTO,1,0.0
3,68770,SUAITA,2023,4,HURTO,1,0.0
4,68406,LEBRIJA,2023,5,HURTO,1,0.0


In [8]:
crime_mensual[crime_mensual['delito_categoria'] == 'DROGA']

,codigo_dane_5d,anio,mes,delito_categoria,cantidad
73,68001,2010,1,DROGA,230
78,68001,2010,2,DROGA,255
83,68001,2010,3,DROGA,242
88,68001,2010,4,DROGA,196
93,68001,2010,5,DROGA,224
...,...,...,...,...,...
30969,68895,2024,6,DROGA,1
30976,68895,2024,10,DROGA,4
30979,68895,2025,2,DROGA,7
30981,68895,2025,3,DROGA,14


##### Tabla pivote

In [9]:
crime_features = crime_mensual.pivot_table(
    index=['codigo_dane_5d', 'anio', 'mes'],
    columns='delito_categoria',
    values='cantidad',
    fill_value=0
).reset_index()

crime_features.columns = [
    'codigo_dane_5d','anio','mes',
    'drogas_total','hurto_total','infancia_total',
    'lesiones_total','sexual_total','vif_total'
]

crime_features['total_delitos_mensual'] = (
    crime_features['drogas_total'] +
    crime_features['hurto_total'] +
    crime_features['infancia_total'] +
    crime_features['lesiones_total'] +
    crime_features['sexual_total'] +
    crime_features['vif_total']
)


In [10]:
crime_features.head()

,codigo_dane_5d,anio,mes,drogas_total,hurto_total,infancia_total,lesiones_total,sexual_total,vif_total,total_delitos_mensual
0,68001,2003,1,0.0,221.0,0.0,0.0,0.0,0.0,221.0
1,68001,2003,2,0.0,220.0,0.0,0.0,0.0,0.0,220.0
2,68001,2003,3,0.0,229.0,0.0,0.0,0.0,0.0,229.0
3,68001,2003,4,0.0,175.0,0.0,0.0,0.0,0.0,175.0
4,68001,2003,5,0.0,257.0,0.0,0.0,0.0,0.0,257.0


#### Policía y frentes de seguridad (agregado anual)

In [11]:
direct_policial = pd.read_csv('/content/drive/MyDrive/atlas-del-crimen/raw/directorio_policial.csv')
direct_policial['codigo_dane_5d'] = direct_policial['codigo_dane_5d'].astype(str)
num_unidades_municipio = direct_policial.groupby('codigo_dane_5d').size().reset_index(name='num_unidades')
num_unidades_municipio.head()

,codigo_dane_5d,num_unidades
0,68013,1
1,68020,1
2,68051,1
3,68077,4
4,68079,1


In [12]:
frentes_policia = pd.read_csv('/content/drive/MyDrive/atlas-del-crimen/raw/frentes_policiales.csv')
frentes_policia['codigo_dane_5d'] = frentes_policia['codigo_dane_5d'].astype(str)
frentes_policia_feat = frentes_policia.groupby('codigo_dane_5d').agg(
    frentes_total=('estado','count'),
    frentes_activos=('estado', lambda x: (x=='ACTIVO').sum()),
    integrantes_total=('nro_integrantes','sum')
).reset_index()
frentes_policia_feat.head()

,codigo_dane_5d,frentes_total,frentes_activos,integrantes_total
0,68013.0,3,3,31
1,68020.0,4,4,25
2,68051.0,3,3,43
3,68077.0,11,11,161
4,68079.0,5,5,40


In [13]:
frentes_policia_feat['codigo_dane_5d'] = (
    frentes_policia_feat['codigo_dane_5d']
    .astype(str)
    .str.replace(r'.0$', '', regex=True)  # quitar ".0"
    .str.zfill(5)
)
frentes_policia_feat

,codigo_dane_5d,frentes_total,frentes_activos,integrantes_total
0,68013,3,3,31
1,68020,4,4,25
2,68051,3,3,43
3,68077,11,11,161
4,68079,5,5,40
...,...,...,...,...
72,68861,7,7,50
73,68867,3,3,24
74,68872,3,3,35
75,68895,6,6,68


#### Datos sociales/demográficos/climáticos

In [14]:
pob = pd.read_csv('/content/drive/MyDrive/atlas-del-crimen/raw/demografia.csv')
pob = pob[['codigo_dane_5d','anio','p_cabecera','p_rural','p_total']]
pob.head()

,codigo_dane_5d,anio,p_cabecera,p_rural,p_total
0,68013,2005,282,1778,2060
1,68013,2006,282,1755,2037
2,68013,2007,281,1722,2003
3,68013,2008,281,1722,2003
4,68013,2009,285,1732,2017


In [15]:
colegios = pd.read_csv('/content/drive/MyDrive/atlas-del-crimen/raw/centros_educativos.csv')
colegios['codigo_dane_5d'] = colegios['codigo_dane'].astype(str).str[:5]

col_feat = colegios.groupby('codigo_dane_5d').agg(
    num_colegios=('nombre_colegio','count'),
    num_publicos=('sector', lambda x: (x=='OFICIAL').sum()),
    num_privados=('sector', lambda x: (x=='PRIVADO').sum())
).reset_index()
col_feat.head()

,codigo_dane_5d,num_colegios,num_publicos,num_privados
0,68001,290,49,241
1,68013,2,1,1
2,68020,4,3,1
3,68051,4,3,1
4,68077,19,2,17


In [16]:
biblio = pd.read_csv('/content/drive/MyDrive/atlas-del-crimen/raw/bibliotecas.csv')
biblio['codigo_dane_5d'] = biblio['codigo_dane'].astype(str).str[:5]
biblio_feat = biblio.groupby('codigo_dane_5d').size().reset_index(name='num_bibliotecas')
biblio_feat.head()

,codigo_dane_5d,num_bibliotecas
0,68001,2
1,68013,1
2,68020,1
3,68051,1
4,68077,1


In [17]:
movil = pd.read_csv('/content/drive/MyDrive/atlas-del-crimen/raw/cobertura_movil.csv')
movil['codigo_dane_5d'] = movil['codigo_dane'].astype(str).str[:5]
movil.head()

,municipio,codigo_dane,latitud,longitud,4g_urbano,3g_urbano,ops_urbano,4g_rural,3g_rural,ops_rural,brecha_4g,codigo_dane_5d
0,BUCARAMANGA,68001,7.116470,-73.132562,1,1,4,1,1,2,0,68001
1,AGUADA,68013,6.162355,-73.523132,1,1,4,0,0,0,1,68013
2,ALBANIA,68020,5.759166,-73.913360,1,1,4,1,1,1,0,68020
3,ARATOCA,68051,6.694418,-73.017860,1,1,4,1,1,3,0,68051
4,BARBOSA,68077,5.932531,-73.615965,1,1,4,1,1,1,0,68077


In [18]:
clima = pd.read_csv('/content/drive/MyDrive/atlas-del-crimen/raw/precipitaciones.csv')
clima['codigo_dane_5d'] = clima['codigo_dane'].astype(str).str[:5]

clima['fecha'] = pd.to_datetime(clima['fecha'])
clima['anio'] = clima['fecha'].dt.year
clima['mes'] = clima['fecha'].dt.month

clima_feat = clima.groupby(['codigo_dane_5d','anio','mes']).agg(
    lluvia_prom=('precipitacion_mm','mean'),
    lluvia_total=('precipitacion_mm','sum')
).reset_index()
clima.head()

,municipio,codigo_dane,fecha,precipitacion_mm,latitud,longitud,codigo_dane_5d,anio,mes
0,ARATOCA,68051,2017-02-27,0.0,6.694418,-73.01786,68051,2017,2
1,ARATOCA,68051,2017-03-02,0.0,6.694418,-73.01786,68051,2017,3
2,ARATOCA,68051,2017-03-03,0.0,6.694418,-73.01786,68051,2017,3
3,ARATOCA,68051,2017-03-04,0.0,6.694418,-73.01786,68051,2017,3
4,ARATOCA,68051,2017-03-05,0.0,6.694418,-73.01786,68051,2017,3


### Unión final de los datos

In [19]:
for tabla in [
    crime_features,
    pob,
    num_unidades_municipio,
    frentes_policia_feat,
    col_feat,
    biblio_feat,
    movil,
    clima_feat
]:
    tabla['codigo_dane_5d'] = tabla['codigo_dane_5d'].astype(str)


In [20]:
df = crime_features.merge(num_unidades_municipio, on='codigo_dane_5d', how='left')
df = df.merge(frentes_policia_feat, on='codigo_dane_5d', how='left')
df = df.merge(pob, on=['codigo_dane_5d','anio'], how='left')
df = df.merge(col_feat, on='codigo_dane_5d', how='left')
df = df.merge(biblio_feat, on='codigo_dane_5d', how='left')
df = df.merge(movil, on='codigo_dane_5d', how='left')
df = df.merge(clima_feat, on=['codigo_dane_5d','anio','mes'], how='left')

In [21]:
df

,codigo_dane_5d,anio,mes,drogas_total,hurto_total,infancia_total,lesiones_total,sexual_total,vif_total,total_delitos_mensual,...,longitud,4g_urbano,3g_urbano,ops_urbano,4g_rural,3g_rural,ops_rural,brecha_4g,lluvia_prom,lluvia_total
0,68001,2003,1,0.0,221.0,0.0,0.0,0.0,0.0,221.0,...,-73.132562,1,1,4,1,1,2,0,NaN,NaN
1,68001,2003,2,0.0,220.0,0.0,0.0,0.0,0.0,220.0,...,-73.132562,1,1,4,1,1,2,0,NaN,NaN
2,68001,2003,3,0.0,229.0,0.0,0.0,0.0,0.0,229.0,...,-73.132562,1,1,4,1,1,2,0,NaN,NaN
3,68001,2003,4,0.0,175.0,0.0,0.0,0.0,0.0,175.0,...,-73.132562,1,1,4,1,1,2,0,NaN,NaN
4,68001,2003,5,0.0,257.0,0.0,0.0,0.0,0.0,257.0,...,-73.132562,1,1,4,1,1,2,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13055,68895,2025,2,7.0,1.0,0.0,0.0,0.0,0.0,8.0,...,-73.268034,1,1,4,1,1,3,0,NaN,NaN
13056,68895,2025,3,14.0,0.0,2.0,0.0,0.0,0.0,16.0,...,-73.268034,1,1,4,1,1,3,0,NaN,NaN
13057,68895,2025,4,1.0,1.0,0.0,0.0,0.0,1.0,3.0,...,-73.268034,1,1,4,1,1,3,0,NaN,NaN
13058,68895,2025,5,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,-73.268034,1,1,4,1,1,3,0,NaN,NaN


In [22]:
output_dir = '/content/drive/MyDrive/atlas-del-crimen/processed'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
df.to_csv(os.path.join(output_dir, 'processed_data.csv'), index=False)

### Extracción de features temporales

In [23]:
output_dir = '/content/drive/MyDrive/atlas-del-crimen/processed'
df = pd.read_csv(os.path.join(output_dir, 'processed_data.csv'))

In [24]:
df_filtered = df[~df['anio'].isin([2003, 2004])].copy()
df_filtered.head()

,codigo_dane_5d,anio,mes,drogas_total,hurto_total,infancia_total,lesiones_total,sexual_total,vif_total,total_delitos_mensual,...,longitud,4g_urbano,3g_urbano,ops_urbano,4g_rural,3g_rural,ops_rural,brecha_4g,lluvia_prom,lluvia_total
24,68001,2005,1,0.0,89.0,0.0,0.0,0.0,0.0,89.0,...,-73.132562,1,1,4,1,1,2,0,NaN,NaN
25,68001,2005,2,0.0,136.0,0.0,0.0,0.0,0.0,136.0,...,-73.132562,1,1,4,1,1,2,0,NaN,NaN
26,68001,2005,3,0.0,122.0,0.0,0.0,0.0,0.0,122.0,...,-73.132562,1,1,4,1,1,2,0,NaN,NaN
27,68001,2005,4,0.0,132.0,0.0,0.0,0.0,0.0,132.0,...,-73.132562,1,1,4,1,1,2,0,NaN,NaN
28,68001,2005,5,0.0,150.0,0.0,0.0,0.0,0.0,150.0,...,-73.132562,1,1,4,1,1,2,0,NaN,NaN


In [25]:
df_filtered['mes_sin'] = np.sin(2*np.pi*df_filtered['mes']/12)
df_filtered['mes_cos'] = np.cos(2*np.pi*df_filtered['mes']/12)

df_filtered = df_filtered.sort_values(['codigo_dane_5d','anio','mes'])
df_filtered['lag_1'] = df_filtered.groupby('codigo_dane_5d')['total_delitos_mensual'].shift(1)
df_filtered['lag_3'] = df_filtered.groupby('codigo_dane_5d')['total_delitos_mensual'].shift(3)
df_filtered['rolling_3'] = df_filtered.groupby('codigo_dane_5d')['total_delitos_mensual'].rolling(3).mean().reset_index(drop=True)

df_filtered = df_filtered.dropna(subset=['lag_1', 'lag_3', 'rolling_3'])
df_filtered = df_filtered.fillna(0)
df_filtered = df_filtered.reset_index(drop=True)

In [26]:
#CREAR EL TARGET LOGARÍTMICO
df_filtered['total_log'] = np.log1p(df_filtered['total_delitos_mensual'])
df_filtered['drogas_log'] = np.log1p(df_filtered['drogas_total'])
df_filtered['hurto_total_log'] = np.log1p(df_filtered['hurto_total'])
df_filtered['infancia_total_log'] = np.log1p(df_filtered['infancia_total'])
df_filtered['lesiones_total_log'] = np.log1p(df_filtered['lesiones_total'])
df_filtered['sexual_total_log'] = np.log1p(df_filtered['sexual_total'])
df_filtered['vif_total_log'] = np.log1p(df_filtered['vif_total'])

In [27]:
df_filtered

,codigo_dane_5d,anio,mes,drogas_total,hurto_total,infancia_total,lesiones_total,sexual_total,vif_total,total_delitos_mensual,...,lag_1,lag_3,rolling_3,total_log,drogas_log,hurto_total_log,infancia_total_log,lesiones_total_log,sexual_total_log,vif_total_log
0,68001,2005,4,0.0,132.0,0.0,0.0,0.0,0.0,132.0,...,122.0,89.0,210.333333,4.890349,0.0,4.890349,0.0,0.0,0.000000,0.000000
1,68001,2005,5,0.0,150.0,0.0,0.0,0.0,0.0,150.0,...,132.0,136.0,187.333333,5.017280,0.0,5.017280,0.0,0.0,0.000000,0.000000
2,68001,2005,6,0.0,171.0,0.0,0.0,0.0,0.0,171.0,...,150.0,122.0,170.333333,5.147494,0.0,5.147494,0.0,0.0,0.000000,0.000000
3,68001,2005,7,0.0,256.0,0.0,0.0,0.0,0.0,256.0,...,171.0,132.0,175.333333,5.549076,0.0,5.549076,0.0,0.0,0.000000,0.000000
4,68001,2005,8,0.0,270.0,0.0,0.0,0.0,0.0,270.0,...,256.0,150.0,165.666667,5.602119,0.0,5.602119,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11879,68867,2016,8,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,1.0,4.000000,0.693147,0.0,0.000000,0.0,0.0,0.693147,0.000000
11880,68867,2016,9,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,2.0,8.666667,0.693147,0.0,0.693147,0.0,0.0,0.000000,0.000000
11881,68867,2016,12,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,1.0,9.000000,0.693147,0.0,0.000000,0.0,0.0,0.000000,0.693147
11882,68867,2017,1,0.0,0.0,0.0,0.0,0.0,2.0,2.0,...,1.0,1.0,6.666667,1.098612,0.0,0.000000,0.0,0.0,0.000000,1.098612


### Entrenamiento

In [28]:
X = df_filtered.copy()

# Eliminar columnas que NO deben entrenarse
columnas_no_modelo = [
    'municipio',
    'codigo_dane',
    'codigo_dane_5d',
    'latitud',
    'longitud',
    'total_delitos_mensual',  # target
    'total_log' # target_log
    'drogas_log',
    'hurto_total_log',
    'infancia_total_log',
    'lesiones_total_log',
    'sexual_total_log',
    'vif_total_log'
]

X = X.drop(columns=columnas_no_modelo, errors='ignore')

# Garantizar solo valores numéricos
X = X.select_dtypes(include=['number'])

# Target
y = df_filtered['total_log']

feature_cols = X.columns.tolist()
print("Features usadas:", feature_cols)

Features usadas: ['anio', 'mes', 'drogas_total', 'hurto_total', 'infancia_total', 'lesiones_total', 'sexual_total', 'vif_total', 'num_unidades', 'frentes_total', 'frentes_activos', 'integrantes_total', 'p_cabecera', 'p_rural', 'p_total', 'num_colegios', 'num_publicos', 'num_privados', 'num_bibliotecas', '4g_urbano', '3g_urbano', 'ops_urbano', '4g_rural', '3g_rural', 'ops_rural', 'brecha_4g', 'lluvia_prom', 'lluvia_total', 'mes_sin', 'mes_cos', 'lag_1', 'lag_3', 'rolling_3', 'total_log', 'drogas_log']


In [29]:
mask_train = df_filtered['anio'] <= 2021

X_train = X[mask_train]
X_test  = X[~mask_train]

y_train = y[mask_train]
y_test  = y[~mask_train]

print("X_train:", X_train.shape)
print("X_test :", X_test.shape)

X_train: (9332, 35)
X_test : (2552, 35)


In [31]:
from xgboost import XGBRegressor

model = XGBRegressor(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8
)

model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=300,
             n_jobs=None, num_parallel_tree=None, ...)

### Evaluación

In [38]:
pred = model.predict(X_test)

print("MAE :", mean_absolute_error(y_test, pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, pred)))
print("R2  :", r2_score(y_test, pred))

MAE : 0.01810861294267633
RMSE: 0.041351666612275566
R2  : 0.9992211346906289


#### Backtesting

In [39]:
pred_log_test = model.predict(X_test)
y_pred_test = np.expm1(pred_log_test)   # Importante invertir el dato logarímico
real_test  = np.expm1(y_test)           # convertir y_test también

In [40]:
resultados = df.loc[X_test.index, ['codigo_dane_5d','anio','mes']].copy()
resultados['real'] = real_test
resultados['pred'] = y_pred_test
resultados

,codigo_dane_5d,anio,mes,real,pred
190,68001,2019,10,969.0,889.963135
191,68001,2019,11,957.0,881.866516
192,68001,2019,12,1111.0,1043.807251
193,68001,2020,1,947.0,882.056396
194,68001,2020,2,1059.0,944.991760
...,...,...,...,...,...
11840,68773,2018,4,2.0,1.957098
11841,68773,2018,5,1.0,0.992037
11842,68773,2018,6,14.0,14.170123
11843,68773,2018,7,1.0,1.084116


### Guardar modelo

In [ ]:
output_model_dir = '/content/drive/MyDrive/atlas-del-crimen/models'
output_model = os.path.join(output_model_dir, 'modelo_total_delitos_log.json')
output_model_features = os.path.join(output_model_dir, 'modelo_total_delitos_log_features.json')
model.save_model(os.path.join(output_model))

In [ ]:
with open(output_model_features, "w") as f:
    json.dump(feature_cols, f)